In [18]:
# Import necessary library
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np 
import joblib
import os
import yaml

In [19]:
def load_parameter(parameter_direction):
    with open(parameter_direction,'r') as file:
        params = yaml.safe_load(file)
    
    return params

In [20]:
params = load_parameter("config/configuration_file_1.yaml")

In [21]:
params

{'data_direction': 'raw_data/indeks-standar-pencemar-udara-di-spku-bulan-juni-sampai-desember-tahun-2021.xlsx',
 'datetime_columns': ['tanggal'],
 'int32_columns': ['pm10', 'pm25', 'so2', 'co', 'o3', 'no2', 'max'],
 'label': 'categori',
 'label_categories': ['BAIK', 'TIDAK SEHAT'],
 'object_columns': ['stasiun', 'critical', 'categori'],
 'predictors': ['stasiun', 'pm10', 'pm25', 'so2', 'co', 'o3', 'no2'],
 'range_co': [-1, 100],
 'range_no2': [-1, 100],
 'range_o3': [-1, 160],
 'range_pm10': [-1, 800],
 'range_pm25': [-1, 400],
 'range_so2': [-1, 500],
 'range_stasiun': ['DKI1 (Bunderan HI)',
  'DKI2 (Kelapa Gading)',
  'DKI3 (Jagakarsa)',
  'DKI4 (Lubang Buaya)',
  'DKI5 (Kebon Jeruk) Jakarta Barat']}

In [22]:
# Import dataset
dataset = pd.read_excel(params['data_direction'])

In [23]:
# Show dataset
dataset

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
0,2021-06-01,DKI1 (Bunderan HI),59,83,22,18,19,35,83,PM25,SEDANG
1,2021-06-02,DKI1 (Bunderan HI),59,84,21,20,24,38,84,PM25,SEDANG
2,2021-06-03,DKI1 (Bunderan HI),54,76,22,20,17,41,76,PM25,SEDANG
3,2021-06-04,DKI1 (Bunderan HI),63,87,20,13,14,30,87,PM25,SEDANG
4,2021-06-05,DKI1 (Bunderan HI),59,79,23,20,19,38,79,PM25,SEDANG
...,...,...,...,...,...,...,...,...,...,...,...
1065,2021-12-27,DKI5 (Kebon Jeruk) Jakarta Barat,54,76,36,14,21,47,76,PM25,SEDANG
1066,2021-12-28,DKI5 (Kebon Jeruk) Jakarta Barat,44,68,20,11,21,33,68,PM25,SEDANG
1067,2021-12-29,DKI5 (Kebon Jeruk) Jakarta Barat,34,54,28,8,25,29,54,PM25,SEDANG
1068,2021-12-30,DKI5 (Kebon Jeruk) Jakarta Barat,53,75,25,15,23,44,75,PM25,SEDANG


In [24]:
# Sanity check
dataset.shape

(1070, 11)

In [25]:
# simpan dataset yang telah digabungkan
joblib.dump(dataset, "processed/dataset.pkl")

['processed/dataset.pkl']

# Data Pipeline

In [26]:
# Check available column in dataframe
dataset.columns

Index(['tanggal', 'stasiun', 'pm10', 'pm25', 'so2', 'co', 'o3', 'no2', 'max',
       'critical', 'categori'],
      dtype='object')

In [27]:
# Check data type
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1070 entries, 0 to 1069
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1070 non-null   datetime64[ns]
 1   stasiun   1070 non-null   object        
 2   pm10      1070 non-null   object        
 3   pm25      1070 non-null   object        
 4   so2       1070 non-null   object        
 5   co        1070 non-null   object        
 6   o3        1070 non-null   object        
 7   no2       1070 non-null   object        
 8   max       1070 non-null   object        
 9   critical  1061 non-null   object        
 10  categori  1069 non-null   object        
dtypes: datetime64[ns](1), object(10)
memory usage: 92.1+ KB


**1. Data Validation**

In [28]:
def non_numerical_finder(data, columns):
    
    """
    This function is to locate a non numerical value within dataset. 
    
    :param data: <pandas dataframe> data contain sample
    :param column: <string> column name
    :return non_numerical: <list> list contain indexes
    """
    
    # Create blank list
    non_numerical = []
    
    # Loop to find non numerical column
    for col in columns:
        non_numeric = pd.to_numeric(data[col], errors = 'coerce').isna()
        non_numerical.extend(non_numeric[non_numeric == True].index.tolist())
    
    # Remove duplicates
    non_numerical = list(set(non_numerical))
    
    return non_numerical

In [29]:
# Find non numerical index
non_numerical = non_numerical_finder(dataset, params['int32_columns'])

In [30]:
# Show table which row or index have non numerical value
dataset.loc[non_numerical]

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
1025,2021-12-18,DKI4 (Lubang Buaya),---,69,42,13,24,17,69,PM25,SEDANG
515,2021-09-26,DKI2 (Kelapa Gading),56,74,---,7,54,6,74,PM25,SEDANG
1028,2021-12-21,DKI4 (Lubang Buaya),62,---,42,17,13,27,62,PM10,SEDANG
516,2021-09-27,DKI2 (Kelapa Gading),53,77,---,11,47,17,77,PM25,SEDANG
517,2021-09-28,DKI2 (Kelapa Gading),49,71,---,8,49,13,71,PM25,SEDANG
...,...,...,...,...,...,...,...,...,...,...,...
993,2021-12-17,DKI3 (Jagakarsa),39,59,43,13,---,16,59,PM25,SEDANG
1004,2021-12-28,DKI3 (Jagakarsa),51,74,---,13,17,18,74,PM25,SEDANG
1007,2021-12-31,DKI3 (Jagakarsa),64,52,21,44,17,---,64,PM10,SEDANG
1019,2021-12-12,DKI4 (Lubang Buaya),---,---,---,---,---,---,0,NaN,TIDAK ADA DATA


In [31]:
# Replace value function
def replace_value(data, column, column_number, old_value, replace_value):
    
    """
    This function is to locate and replace value from a column. 
    
    :param data: <pandas dataframe> data contain sample
    :param column: <string> column name
    :param column_number: <string> single or multiple column
    :param old_value: <string/int/float> old value or current value
    :param replace_value: <string/int/float> value to replace old value
    :return data: <pandas dataframe> data contain sample
    """
    
    if str(column_number).lower() == "multiple":
        for col in column:
            data[col] = data[col].replace(old_value, replace_value)
    elif str(column_number).lower() == "single":
        data[column] = data[column].replace(old_value, replace_value)
        
    else: 
        "column_number is not correct"
        
    return data

In [32]:
# Handling non numeric data
replace_value(data = dataset, 
              column = params['int32_columns'],
              column_number = "multiple",
              old_value = "---", 
              replace_value = -1)

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
0,2021-06-01,DKI1 (Bunderan HI),59,83,22,18,19,35,83,PM25,SEDANG
1,2021-06-02,DKI1 (Bunderan HI),59,84,21,20,24,38,84,PM25,SEDANG
2,2021-06-03,DKI1 (Bunderan HI),54,76,22,20,17,41,76,PM25,SEDANG
3,2021-06-04,DKI1 (Bunderan HI),63,87,20,13,14,30,87,PM25,SEDANG
4,2021-06-05,DKI1 (Bunderan HI),59,79,23,20,19,38,79,PM25,SEDANG
...,...,...,...,...,...,...,...,...,...,...,...
1065,2021-12-27,DKI5 (Kebon Jeruk) Jakarta Barat,54,76,36,14,21,47,76,PM25,SEDANG
1066,2021-12-28,DKI5 (Kebon Jeruk) Jakarta Barat,44,68,20,11,21,33,68,PM25,SEDANG
1067,2021-12-29,DKI5 (Kebon Jeruk) Jakarta Barat,34,54,28,8,25,29,54,PM25,SEDANG
1068,2021-12-30,DKI5 (Kebon Jeruk) Jakarta Barat,53,75,25,15,23,44,75,PM25,SEDANG


In [33]:
# Show table which row or index have non numerical value
dataset.loc[non_numerical]

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
1025,2021-12-18,DKI4 (Lubang Buaya),-1,69,42,13,24,17,69,PM25,SEDANG
515,2021-09-26,DKI2 (Kelapa Gading),56,74,-1,7,54,6,74,PM25,SEDANG
1028,2021-12-21,DKI4 (Lubang Buaya),62,-1,42,17,13,27,62,PM10,SEDANG
516,2021-09-27,DKI2 (Kelapa Gading),53,77,-1,11,47,17,77,PM25,SEDANG
517,2021-09-28,DKI2 (Kelapa Gading),49,71,-1,8,49,13,71,PM25,SEDANG
...,...,...,...,...,...,...,...,...,...,...,...
993,2021-12-17,DKI3 (Jagakarsa),39,59,43,13,-1,16,59,PM25,SEDANG
1004,2021-12-28,DKI3 (Jagakarsa),51,74,-1,13,17,18,74,PM25,SEDANG
1007,2021-12-31,DKI3 (Jagakarsa),64,52,21,44,17,-1,64,PM10,SEDANG
1019,2021-12-12,DKI4 (Lubang Buaya),-1,-1,-1,-1,-1,-1,0,NaN,TIDAK ADA DATA


In [34]:
# Find non numerical index
non_numerical_new = non_numerical_finder(dataset, params['int32_columns'])
non_numerical_new

[917]

In [35]:
# Creating function to replace value to a specific row and cell
def replace_specific_index(data, index, column, new_value):
    
    """
    This function is created to replace a new value to old value located in specific index and column
    :param data: <pandas dataframe> data contain sample
    :param index: <int> specific index
    :param column: <string> specific column
    :param new_value: <string/int/float> new value to replace old value
    :return dataset: <pandas dataframe> data contain sample
    """
    data.loc[index, column] = new_value
    
    return data

In [36]:
# Replace value
replace_specific_index(data = dataset, index = non_numerical_new[0], column = 'max', new_value = 49)
replace_specific_index(data = dataset, index = non_numerical_new[0], column = 'critical', new_value = 'PM10')
replace_specific_index(data = dataset, index = non_numerical_new[0], column = 'categori', new_value = 'BAIK')

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
0,2021-06-01,DKI1 (Bunderan HI),59,83,22,18,19,35,83,PM25,SEDANG
1,2021-06-02,DKI1 (Bunderan HI),59,84,21,20,24,38,84,PM25,SEDANG
2,2021-06-03,DKI1 (Bunderan HI),54,76,22,20,17,41,76,PM25,SEDANG
3,2021-06-04,DKI1 (Bunderan HI),63,87,20,13,14,30,87,PM25,SEDANG
4,2021-06-05,DKI1 (Bunderan HI),59,79,23,20,19,38,79,PM25,SEDANG
...,...,...,...,...,...,...,...,...,...,...,...
1065,2021-12-27,DKI5 (Kebon Jeruk) Jakarta Barat,54,76,36,14,21,47,76,PM25,SEDANG
1066,2021-12-28,DKI5 (Kebon Jeruk) Jakarta Barat,44,68,20,11,21,33,68,PM25,SEDANG
1067,2021-12-29,DKI5 (Kebon Jeruk) Jakarta Barat,34,54,28,8,25,29,54,PM25,SEDANG
1068,2021-12-30,DKI5 (Kebon Jeruk) Jakarta Barat,53,75,25,15,23,44,75,PM25,SEDANG


In [37]:
# Handling non numeric for pm10, pm25, so2, co, o3, no2
def change_data_type(data, column, data_type):
    
    """
    This function is to change the type of a specific column into a desired type.
    
    :param data: <pandas dataframe> data contain sample
    :param column: <string> column name
    :param data_type: <string> desired data type
    :return dataset: <pandas dataframe> data contain sample
    """
    
    for col in column:
        data[col] = data[col].astype(data_type)
    
    return data

In [38]:
# Change column type
change_data_type(data = dataset, column = params['int32_columns'], data_type = int)

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
0,2021-06-01,DKI1 (Bunderan HI),59,83,22,18,19,35,83,PM25,SEDANG
1,2021-06-02,DKI1 (Bunderan HI),59,84,21,20,24,38,84,PM25,SEDANG
2,2021-06-03,DKI1 (Bunderan HI),54,76,22,20,17,41,76,PM25,SEDANG
3,2021-06-04,DKI1 (Bunderan HI),63,87,20,13,14,30,87,PM25,SEDANG
4,2021-06-05,DKI1 (Bunderan HI),59,79,23,20,19,38,79,PM25,SEDANG
...,...,...,...,...,...,...,...,...,...,...,...
1065,2021-12-27,DKI5 (Kebon Jeruk) Jakarta Barat,54,76,36,14,21,47,76,PM25,SEDANG
1066,2021-12-28,DKI5 (Kebon Jeruk) Jakarta Barat,44,68,20,11,21,33,68,PM25,SEDANG
1067,2021-12-29,DKI5 (Kebon Jeruk) Jakarta Barat,34,54,28,8,25,29,54,PM25,SEDANG
1068,2021-12-30,DKI5 (Kebon Jeruk) Jakarta Barat,53,75,25,15,23,44,75,PM25,SEDANG


In [39]:
# Sanity check
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1070 entries, 0 to 1069
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1070 non-null   datetime64[ns]
 1   stasiun   1070 non-null   object        
 2   pm10      1070 non-null   int64         
 3   pm25      1070 non-null   int64         
 4   so2       1070 non-null   int64         
 5   co        1070 non-null   int64         
 6   o3        1070 non-null   int64         
 7   no2       1070 non-null   int64         
 8   max       1070 non-null   int64         
 9   critical  1061 non-null   object        
 10  categori  1070 non-null   object        
dtypes: datetime64[ns](1), int64(7), object(3)
memory usage: 92.1+ KB


In [40]:
# Categori proportion
dataset[params['label']].value_counts()

SEDANG            791
TIDAK SEHAT       207
BAIK               63
TIDAK ADA DATA      9
Name: categori, dtype: int64

In [41]:
# Handling irrelevant data
dataset.drop(index = dataset[dataset[params['label']] == "TIDAK ADA DATA"].index, inplace = True)

In [42]:
# Categori proportion
dataset[params['label']].value_counts()

SEDANG         791
TIDAK SEHAT    207
BAIK            63
Name: categori, dtype: int64

In [43]:
# Sanity check
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1061 entries, 0 to 1069
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   1061 non-null   datetime64[ns]
 1   stasiun   1061 non-null   object        
 2   pm10      1061 non-null   int64         
 3   pm25      1061 non-null   int64         
 4   so2       1061 non-null   int64         
 5   co        1061 non-null   int64         
 6   o3        1061 non-null   int64         
 7   no2       1061 non-null   int64         
 8   max       1061 non-null   int64         
 9   critical  1061 non-null   object        
 10  categori  1061 non-null   object        
dtypes: datetime64[ns](1), int64(7), object(3)
memory usage: 99.5+ KB


In [44]:
joblib.dump(dataset, "processed/dataset_clean.pkl")

['processed/dataset_clean.pkl']

**2. Data Defense**

In [45]:
def check_data(input_data, params):
    # check data types
    assert input_data.select_dtypes("datetime").columns.to_list() == params["datetime_columns"], "an error occurs in datetime column(s)."
    assert input_data.select_dtypes("object").columns.to_list() == params["object_columns"], "an error occurs in object column(s)."
    assert input_data.select_dtypes("int").columns.to_list() == params["int32_columns"], "an error occurs in int32 column(s)."

    # check range of data
    assert set(input_data.stasiun).issubset(set(params["range_stasiun"])), "an error occurs in stasiun range."
    assert input_data['pm10'].between(params["range_pm10"][0], params["range_pm10"][1]).sum() == len(input_data), "an error occurs in pm10 column."
    assert input_data['pm25'].between(params["range_pm25"][0], params["range_pm25"][1]).sum() == len(input_data), "an error occurs in pm25 column."
    assert input_data['so2'].between(params["range_so2"][0], params["range_so2"][1]).sum() == len(input_data), "an error occurs in so2 column."
    assert input_data['co'].between(params["range_co"][0], params["range_co"][1]).sum() == len(input_data), "an error occurs in co column."
    assert input_data['o3'].between(params["range_o3"][0], params["range_o3"][1]).sum() == len(input_data), "an error occurs in o3 column."
    assert input_data['no2'].between(params["range_no2"][0], params["range_no2"][1]).sum() == len(input_data), "an error occurs in no2 column."

In [46]:
# Check data
check_data(dataset, params)

**3. Data Splitting**

In [47]:
# Input-Output split function
def input_output_split(df, column):
    
    """
    This function is created to split input and output column
    :param df: <pandas dataframe> data contain sample
    :param columns: <string> column name
    :return X: <pandas dataframe> data contain input
    :return y: <pandas dataframe> data contain output
    """
    
    X = df.drop(column, axis = 1)
    y = df[column]
    
    return X,y

In [48]:
X, y = input_output_split(df = dataset, column = params['label'])

In [49]:
# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.35,
                                                    stratify = y,
                                                    random_state = 123)

In [50]:
# Train-Test split
X_valid, X_test, y_valid, y_test = train_test_split(X_test,
                                                    y_test,
                                                    test_size = 0.5,
                                                    stratify = y_test,
                                                    random_state = 123)

In [51]:
joblib.dump(X_train, "processed/X_train.pkl")
joblib.dump(y_train, "processed/y_train.pkl")
joblib.dump(X_valid, "processed/X_valid.pkl")
joblib.dump(y_valid, "processed/y_valid.pkl")
joblib.dump(X_test, "processed/X_test.pkl")
joblib.dump(y_test, "processed/y_test.pkl")

['processed/y_test.pkl']